In [1]:
import urllib.request, urllib.parse, urllib.error                                                                      #importing library
from bs4 import BeautifulSoup                                                                                          #importing library
import requests                                                                                                        #importing library

print("This is a program to scrape data from the https://money.cnn.com/data/hotstocks/ for a class project.\n")        #print command
print("Which stock are you interested in:\n")                                                                          #print command

Stock_type = ["Most Actives", "Gainers","Losers"]                                                                      #creating list of element and assigning it to variable

def lst_of_stocks():                                                                                                   #function defined to create list of stocks and display them 
    try:                                                                                                               #try block starts
        page = requests.get("https://money.cnn.com/data/hotstocks/")                                                   #requesting for the html of a page
    except:                                                                                                            #except block
        print('Webpage or server cannot be found')                                                                     #print command when exception is encountered

    soup = BeautifulSoup(page.content, 'lxml')                                                                         #making soup of the page content
    names = soup.find_all('table', attrs={'class':"wsod_dataTable wsod_dataTableBigAlt"})                              #finding all the tables within the soup
    stocks = []                                                                                                        #creating a list named stocks
    
    for st_type, st_name in zip(Stock_type, names):                                                                    #for loop starts zip the Stock_type, names and iterating over them
        print(st_type + ":")                                                                                           #print the stock type in formatted manner
        
        for data in st_name.find_all('td'):                                                                            #iterating over stock name and finding all table data            
            
            if data.a == None:                                                                                         #if condition check
                continue                                                                                    
            elif data.a != None:                                                                                       #elif condition check and perform below actions
                ticker = data.a.text                                                                                   #fetch the ticker symbol from the anchor tag in table data
                company = data.span.text                                                                               #fetch the company name from the anchor tag in table data  
                print(ticker,company)                                                                                  #print the ticker symbol with company name 
                stocks.append([st_type,ticker,company])                                                                #append the ticker, company name in stocks list
                
        print()                                                                                                        #print command
    return stocks                                                                                                      #return all the stock type and stocks

def usr_ticker_detail(usr_stock):                                                                                      #function defined to create ticker details for user

    try:                                                                                                               #try block starts
        stock_url = requests.get("https://money.cnn.com/quote/quote.html?symb={}".format(usr_stock))                   #request the html of the specified ticker page
    except:                                                                                                            #except block starts
        print('Webpage cannot be found')                                                                               #print if exception is raised
        
    soup = BeautifulSoup(stock_url.content,'lxml')                                                                     #making soup of the page content
    details = soup.find('table', attrs={'class':"wsod_dataTable wsod_dataTableBig"})                                   #finding all the tables within the soup

    Todays_open = details.find("td",text="Today’s open").find_next_sibling("td").string                                #finding the stock open data
    print('OPEN:', Todays_open)                                                                                        #print the open data
    Previous_close = soup.find("td",text="Previous close").find_next_sibling("td").string                              #finding the stock previous close data 
    print('PREV CLOSE:', Previous_close)                                                                               #print the close data
    volume = soup.find("td",text="Volume").find_next_sibling("td").string                                              #finding the stock volume data
    print('VOLUME:', volume)                                                                                           #print the volume data
    Market_cap = soup.find("td",text="Market cap").find_next_sibling("td").string                                      #finding the market cap data
    print('MARKET CAP:', Market_cap)                                                                                   #print the market cap data


def write_file(All_stocks):                                                                                            #function defined to fetch the data of every stock and write them in a file
    
    file_obj = open("Stock.csv","w")                                                                                   #open the file with the specified name in write mode
    
    Head = ['Stock Category', 'Ticker symbol','Company name','OPEN','PREV CLOSE','VOLUME','MARKET CAP']                #creating list of header elements
    Header = ",".join(Head)                                                                                            #joining them with comma and assigning it to a variable
    file_obj.writelines(Header + '\n')                                                                                 #writing the file header

    for data in All_stocks:                                                                                            #iterating over company detail list in list of all stock data
        
        ticker_url = data[1]                                                                                           #taking the tickers out of the whole data from list

        stock_url = requests.get("https://money.cnn.com/quote/quote.html?symb={}".format(ticker_url))                  #request the html for the specified page
        soup = BeautifulSoup(stock_url.content,'lxml')                                                                 #making soup of page content
        details = soup.find('table', attrs={'class':"wsod_dataTable wsod_dataTableBig"})                               #finding the tables within the soup
        
        Todays_open1 = details.find("td",text="Today’s open").find_next_sibling("td").string                           #fetch the todays open value of stock from soup
        Todays_open = Todays_open1.replace(',','')                                                                     #replace the comma in the figure
        data.insert(3,Todays_open)                                                                                     #insert the todays open value at specified position in the stock data list

        Previous_close1 = soup.find("td",text="Previous close").find_next_sibling("td").string                         #fetch the previous close of stock from soup
        Previous_close = Previous_close1.replace(',','')                                                               #replace the comma in the figure
        data.append(Previous_close)                                                                                    #append it in the stock data list

        volume1 = soup.find("td",text="Volume").find_next_sibling("td").string                                         #fetch volume of stock from soup
        volume = volume1.replace(',','')                                                                               #replace the comma in the figure
        data.append(volume)                                                                                            #append it in the stock data list
        
        Market_cap1 = soup.find("td",text="Market cap").find_next_sibling("td").string                                 #fetch market cap detail of stock from soup
        Market_cap = Market_cap1.replace(',','')                                                                       #replace the comma in the figure
        data.append(Market_cap)                                                                                        #append it in the stock data list

        line = ",".join(data)                                                                                          #joining the stock data with comma and assigning it to variable
        file_obj.writelines(line + '\n')                                                                               #writing content in the file    
    file_obj.close()                                                                                                   #closing the file



All_stocks = lst_of_stocks()                                                                                           #storing list of all stock in a variable


write_file(All_stocks)                                                                                                 #calling a function to write data in file


try:                                                                                                                   #try block starts
    usr_stock = input("User inputs: ").upper()                                                                         #get the input from user and conver it to uppercase
except:                                                                                                                #except block
    print('Invalid input.')                                                                                            #print when invalid input is entered


for data in All_stocks:                                                                                                #iterating over the list of all stock
    if usr_stock == data[1]:                                                                                           #check if the user's input is among one of the ticker symbol 
        print("\nThe data of {} {} is the following:".format(usr_stock,data[2]))                                       #print the line
        print(usr_stock,data[2])                                                                                       #print the stock ticker with company name
        usr_ticker_detail(usr_stock)                                                                                   #calling a function to fetch the user stock details
        break
if len(usr_stock) == 0 or usr_stock != data[1]:                                                                        #condition check for user's input
    print('Invalid stock ticker input.')                                                                               #print when entered ticker is other than the listed ones

This is a program to scrape data from the https://money.cnn.com/data/hotstocks/ for a class project.

Which stock are you interested in:

Most Actives:
CCL Carnival Corp
F Ford Motor Co
NCLH Norwegian Cruise Line Holdings Ltd
T AT&T Inc
BAC Bank of America Corp
PFE Pfizer Inc
C Citigroup Inc
VZ Verizon Communications Inc
DAL Delta Air Lines Inc
KR Kroger Co

Gainers:
NCLH Norwegian Cruise Line Holdings Ltd
CCL Carnival Corp
RCL Royal Caribbean Cruises Ltd
LVS Las Vegas Sands Corp
CTLT Catalent Inc
NVR NVR Inc
EMN Eastman Chemical Co
CTRA Coterra Energy Inc
TWTR Twitter Inc
SWK Stanley Black & Decker Inc

Losers:
KR Kroger Co
BFB Brown-Forman Corp
TGT Target Corp
PKG Packaging Corp of America
DG Dollar General Corp
K Kellogg Co
STT State Street Corp
PGR Progressive Corp
AZO Autozone Inc
BDX Becton Dickinson and Co

User inputs: RCL

The data of RCL Royal Caribbean Cruises Ltd is the following:
RCL Royal Caribbean Cruises Ltd
OPEN: 73.21
PREV CLOSE: 71.64
VOLUME: 7,703,121
MARKET CAP: $1